In [2]:
import pandas as pd
import numpy as np
import math
import datetime
import itertools
import warnings
import pandas as pd
import numpy as np
import math
import datetime
from ipynb.fs.defs.functions import new_route
from ipynb.fs.defs.functions import get_row
from ipynb.fs.defs.functions import get_column
from ipynb.fs.defs.functions import getName
from ipynb.fs.defs.functions import timeDistance
from ipynb.fs.defs.functions import best_index
from ipynb.fs.defs.functions import find_best_position
from ipynb.fs.defs.functions import cities_list
warnings.filterwarnings('ignore')

table=pd.read_csv("../Dati/orari/route.csv", delimiter=";")
time_table=pd.read_csv("../Dati/orari/route_time.csv", delimiter=";")
dft = pd.read_csv("../Dati/distanze/times_computed.csv", delimiter=";", encoding = "ISO-8859-1", index_col=0)
dfd = pd.read_csv("../Dati/distanze/distances_computed.csv", delimiter=";", encoding = "ISO-8859-1", index_col=0)

In [3]:
#(inizio e fine compresi)
#Ritorna una lista contenente le chiavi di due città e il tempo di percorrenza tra le due
#nel formato (times, "key1 - key2")
#"key1 - key2" è la chiave
#La lista partirà dalla posizione "inizio" della schedulazione e finità con "fine"
def route(schedule, inizio, fine, dft):
    times = []
    keys = []
    row = dft[get_row(dft, schedule[inizio]).values]
    key1 = ''
    key2 = ''
    impossibile = []
    
    for i in range(inizio+1,fine+1):
        key1 = row.index[0]
        try:
            paese=schedule[i]
            if 'Martignana' in paese:
                paese = 'Martignana di Po'
        except:
            continue
        #Distanze in tempo dal paese corrente di ognuno dei furgoni alla destinazione
        value=row.loc[:,get_column(dft, paese).values].values[0][0]
        times.append(round(value))
        row = dft[get_row(dft, paese).values]
        key2 = row.index[0]
        keys.append(key1 + ' $ ' + key2)
    
    steps_times1 = pd.Series(times , keys)
    return steps_times1

In [4]:
#Datogli una schedulazione calcola il tempo di percorrenza
def schedule_time(schedule, dft):
    times = []
    row = dft[get_row(dft, 'Cremona, sesto 39').values]

    for i in range(1, len(schedule)):
        paese=schedule[i]
        if 'Martignana' in paese:
            paese = 'Martignana di Po'

        value=row.loc[:,get_column(dft, paese).values].values[0][0]
        times.append(round(value))
        
        row = dft[get_row(dft, paese).values]
    return sum(times)

In [5]:
#mantiene il nodo inizial e il successivo a y invariati, mentre fa il reverse della lista tra i due
#A-b-c-d-E -> A-d-c-b-E
def cross(schedule, x, y):
    i = [schedule[x]]
    j1 = [schedule[y+1]]
    j1_i1 = list(schedule[x+1: y+1].sort_index(ascending = False).values) 
    return i + j1_i1 + j1

In [6]:
#Data una schedulazione e due nodi di cui fare l'incrocio dei nodi
#effettua la tabulazione del nuovo percorso, dopodichè restituisce il migliore
#tra il nuovo e il vecchio
def two_opt(schedule , i, j, dft):
    #computo la prima parte che rimane uguale
    first_part = route(schedule, 0, i, dft)

    #stesso per l'ultima
    final_part = route(schedule, j+1, len(schedule), dft)
    #print(final_part)
    crossed = cross(schedule, i, j)
    central_part = route(crossed, 0,len(crossed), dft)
    
    new_schedule = first_part.append(central_part).append(final_part)
    if(schedule_time(schedule, dft)> sum(new_schedule.values)):
        return new_schedule
    else:
        return route(schedule, 0, len(schedule), dft)

In [7]:
#TIME 2-OPT
week= ['lunedi', 'martedi', 'mercoledi', 'giovedi', 'venerdi']
fur1 = pd.read_csv("../Dati/turni_f/fine_optimized_start/foss_ls1.csv", delimiter=";", encoding = "ISO-8859-1")
df = pd.DataFrame({})
for day in week:
    print(day)
    #Otteniamo la schedulazione unendo Cremona-tabella del giorno-Ostiano
    #schedule = pd.Series('Cremona, sesto 39')
    schedule = fur1[day].dropna()
    #schedule = schedule.append(pd.Series('Ostiano, Mazzini, 1' , [len(schedule)]))
    #print(schedule)
    #Otteniamo la schedulazione filtrando i nan e salviamo anche il suo tempo di percorrenza
    best_schedule = pd.Series(schedule.dropna())
    best_time = schedule_time(schedule, dft)
    local_best_time = best_time
    #Per ogni i e j effettua il 2-opt e salva se il risultato migliora con lo scambio
    #Alla fine, avendo provato a scambiare tutte le coppie e avendole confrontate otteremo la miglior schedulazione
    #per ogni giro di 2-opt parte dalla schedulazione greedy
    while(True):
        for i in range(1, len(schedule)-2):
            for j in range(i+1, len(schedule)-1):
                s = two_opt(pd.Series(schedule), i, j, dft)
                t = sum(s.values)
                if schedule_time(best_schedule, dft)> t:
                    best_time = t.copy()
                    best_schedule = cities_list(s) 
        if(local_best_time>best_time):
            schedule = best_schedule[:]
            local_best_time = best_time
        else:
            break
    df = pd.concat([df, pd.DataFrame(pd.Series(best_schedule).rename(day))], axis=1)       
df.to_csv('../Dati/turni_f/fine_optimized_start/foss_2opt1.csv', index=False, sep=";", encoding = "ISO-8859-1")

lunedi
martedi
mercoledi
giovedi
venerdi


In [ ]:
week= ['martedi', 'mercoledi', 'giovedi']
fur2 = pd.read_csv("../Dati/turni_f/fine_optimized_start/foss_ls2.csv", delimiter=";", encoding = "ISO-8859-1")
df = pd.DataFrame({})
for day in week:
    print(day)
    #schedule = pd.Series('Cremona, sesto 39')
    schedule = fur2[day].dropna()
    #schedule = schedule.append(pd.Series('San Giovanni in Croce, Feudatari, 25 A' , [len(schedule)]))
    #print(schedule)
    best_schedule = pd.Series(schedule.dropna())
    best_time = schedule_time(schedule, dft)
    local_best_time = best_time
    while(True):
        for i in range(1, len(schedule)-2):
            for j in range(i+1, len(schedule)-1):
                s = two_opt(pd.Series(schedule), i, j, dft)
                t = sum(s.values)
                if schedule_time(best_schedule, dft)> t:
                    best_time = t.copy()
                    best_schedule = cities_list(s) 
        if(local_best_time>best_time):
            schedule = best_schedule[:]
            local_best_time = best_time
        else:
            break 
    df = pd.concat([df, pd.DataFrame(pd.Series(best_schedule).rename(day))], axis=1)        
df.to_csv('../Dati/turni_f/fine_optimized_start/foss_2opt2.csv', index=False, sep=";", encoding = "ISO-8859-1")

In [11]:
#KM 2-OPT
week= ['lunedi', 'martedi', 'mercoledi', 'giovedi', 'venerdi']
df = pd.DataFrame({})
for day in week:
    print(day)
    #Otteniamo la schedulazione unendo Cremona-tabella del giorno-Ostiano
    #schedule = pd.Series('Cremona, sesto 39')
    schedule = fur1d[day].dropna()
    #schedule = schedule.append(pd.Series('Ostiano' , [len(schedule)]))
    #print(schedule)
    #Otteniamo la schedulazione filtrando i nan e salviamo anche il suo tempo di percorrenza
    best_schedule = pd.Series(schedule.dropna())
    best_time = schedule_time(schedule, dfd)
    local_best_time = best_time
    #Per ogni i e j effettua il 2-opt e salva se il risultato migliora con lo scambio
    #Alla fine, avendo provato a scambiare tutte el coppie e avendole confrontate otteremo la miglior schedulazione
    #per ogni giro di 2-opt parte dalla schedulazione greedy
    while(True):
        for i in range(1, len(schedule)-2):
            for j in range(i+1, len(schedule)-1):
                s = two_opt(pd.Series(schedule), i, j, dfd)
                t = sum(s.values)
                if schedule_time(best_schedule, dfd)> t:
                    best_time = t.copy()
                    best_schedule = cities_list(s) 
        if(local_best_time>best_time):
            schedule = best_schedule[:]
            local_best_time = best_time
        else:
            break
    df = pd.concat([df, pd.DataFrame(pd.Series(best_schedule).rename(day))], axis=1)       
df.to_csv('../Dati/turni_f/optimized_start/oss_2opt_km_f1.csv', index=False, sep=";", encoding = "ISO-8859-1")

lunedi
martedi
mercoledi
giovedi
venerdi


In [12]:
week= ['martedi', 'mercoledi', 'giovedi']
df = pd.DataFrame({})
for day in week:
    print(day)
    #schedule = pd.Series('Cremona, sesto 39')
    schedule = fur2d[day].dropna()
    #schedule = schedule.append(pd.Series('San Giovanni in Croce, Feudatari, 25 A' , [len(schedule)]))
    #print(schedule)
    best_schedule = pd.Series(schedule.dropna())
    best_time = schedule_time(schedule, dfd)
    local_best_time = best_time
    while(True):
        for i in range(1, len(schedule)-2):
            for j in range(i+1, len(schedule)-1):
                s = two_opt(pd.Series(schedule), i, j, dfd)
                t = sum(s.values)
                if schedule_time(best_schedule, dfd)> t:
                    best_time = t.copy()
                    best_schedule = cities_list(s) 
        if(local_best_time>best_time):
            schedule = best_schedule[:]
            local_best_time = best_time
        else:
            break 
    df = pd.concat([df, pd.DataFrame(pd.Series(best_schedule).rename(day))], axis=1)        
df.to_csv('../Dati/turni_f/optimized_start/oss_2opt_km_f2.csv', index=False, sep=";", encoding = "ISO-8859-1")

martedi
mercoledi
giovedi
